In [1]:
import pandas as pd

# تحميل البيانات
df = pd.read_csv("cleaned_augmented_success_prediction_dataset.csv")

# حذف الصفوف اللي فيها missing في العمود الهدف
df_clean = df.dropna(subset=["is_successful"]).copy()


***Feature Engineering***

In [2]:
# حساب عمر الشركة بدل سنة التأسيس
current_year = 2025
df_clean["startup_age"] = current_year - df_clean["founded_year"]

# حذف العمود القديم
df_clean.drop("founded_year", axis=1, inplace=True)

In [3]:
# الأعمدة الاسمية اللي هنعملها Label Encoding
from sklearn.preprocessing import LabelEncoder

categorical_cols = ["city", "industry", "startup_stage",
                    "market_competition_level", "num_direct_competitors",
                    "market_need_level"]

label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col])
    label_encoders[col] = le

***Data Splitting (X AND Y)***

In [4]:
# فصل البيانات
X = df_clean.drop("is_successful", axis=1)
y = df_clean["is_successful"]


***Scaling***

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


***Data Splitting (Train / Test)***

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)


***Model Building***

In [7]:
pip install xgboost


In [8]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# تعريف الموديلات
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "KNN": KNeighborsClassifier()
}

# تخزين النتائج
results = {}

# تجربة وتقييم كل موديل
for name, model in models.items():
    print(f"\n====== {name} ======")

    start_time = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    end_time = time.time()

    accuracy = accuracy_score(y_test, y_pred)
    f1 = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
    conf = confusion_matrix(y_test, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf)
    print(f"Time taken: {end_time - start_time:.2f} seconds")

    # تخزين النتائج لاستخدام لاحق
    results[name] = {
        "Accuracy": accuracy,
        "F1 Score": f1,
        "Time": end_time - start_time,
        "Confusion Matrix": conf
    }



====== RandomForest ======
Accuracy: 1.0000
F1 Score: 1.0000
Confusion Matrix:
[[52  0]
 [ 0 28]]
Time taken: 1.08 seconds

====== LogisticRegression ======
Accuracy: 0.8250
F1 Score: 0.8263
Confusion Matrix:
[[44  8]
 [ 6 22]]
Time taken: 0.16 seconds

====== SVM ======
Accuracy: 0.9625
F1 Score: 0.9626
Confusion Matrix:
[[50  2]
 [ 1 27]]
Time taken: 0.03 seconds

====== XGBoost ======
Accuracy: 1.0000
F1 Score: 1.0000
Confusion Matrix:
[[52  0]
 [ 0 28]]
Time taken: 0.15 seconds

====== KNN ======
Accuracy: 0.9625
F1 Score: 0.9629
Confusion Matrix:
[[49  3]
 [ 0 28]]
Time taken: 0.01 seconds


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:38:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


***Use Logistic regression model***

***calculate Training Accuracy and Testing Accuracy***

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

# تدريب الموديل
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# حساب Training Accuracy
train_preds = logreg.predict(X_train)
train_acc = accuracy_score(y_train, train_preds)
print("Training Accuracy:", train_acc)

# حساب Testing Accuracy
test_preds = logreg.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print("Testing Accuracy:", test_acc)


Training Accuracy: 0.840625
Testing Accuracy: 0.825


***Save model as PKL***

In [10]:
# حفظ الموديل
joblib.dump(logreg, "logistic_model.pkl")


['logistic_model.pkl']

In [11]:
import joblib

# 1. حفظ الموديل
joblib.dump(logreg, "logistic_model.pkl")

# 2. حفظ الـ Scaler
joblib.dump(scaler, "scaler.pkl")

# 3. حفظ الـ Label Encoders (لو كنتِ استخدمتيهم في Feature Engineering)
joblib.dump(label_encoders, "label_encoders.pkl")

print("✅ تم حفظ الموديل وكل الأدوات بنجاح!")


✅ تم حفظ الموديل وكل الأدوات بنجاح!
